In [ ]:
import expipe
import pathlib
import quantities as pq
import shutil
import pandas as pd

# Making a project

When making a project we require with a path to where we want the project to reside

In [ ]:
project = expipe.require_project('multimodal_rat_experiments')

## Adding user info in a module

In [ ]:
project.modules['user_info'] = {
    'Mikkel': {
        'age': 34,
        'sex': 'M'
    },
    'Sarah': {
        'age': 43,
        'sex': 'M'
    }
}

In [ ]:
project.modules

In [ ]:
project.modules['user_info']['Sarah']['sex'] = 'F'

# Entity

In [ ]:
rat_180 = project.create_entity('rat_180')

In [ ]:
rat_180.users =  ['Mikkel', 'Sarah']
rat_180.type = 'rat'
rat_180.tags = ['wild type']

In [ ]:
project.entities

In [ ]:
rat_180.tags

## Assignments - add an entity called "rat_181"

In [ ]:
project.entities['rat_181'] = {
    'users': ['Mikkel', 'Sarah'],
    'type': 'rat',
    'tags': ['PVcre']
}

In [ ]:
project.entities

## Adding a module with template

It can be tedious to add all the keys and according information every time we make a module, therefore we use templates. In order to use templates we must add our desired templates to project_path/templates.

In [ ]:
rat_180.modules['info'] = {
    'weight': 500 * pq.g,
    'strain': 'Rattus norvegicus'
}

In [ ]:
rat_180.delete_module('info')

The template "subject" contains all fields that we should fill out when registering an entity/subject

In [ ]:
rat_180.create_module('info', template='subject')

In [ ]:
rat_180.modules['info'].contents

# Action

Now we are ready to start adding actions

## Adding Surgery

The experiments starts with a surgery where we implant an optic fibre and inject a virus containing channel rhodopsin

In [ ]:
project.actions['180_surgery'] = {
    'type': 'Surgery',
    'entities': ['rat_180'],
    'users': ['Mikkel'],
    'tags': ['virus injection', 'implantation']
}

Too see which templates are available we invoke the templates browser

In [ ]:
project.templates

In [ ]:
project.actions['180_surgery'].create_module(template='anaesthesia_isoflurane')

In [ ]:
project.actions['180_surgery'].create_module(template='injection_ms')

In [ ]:
project.actions['180_surgery'].create_module(template='implant_drive_mecr')
project.actions['180_surgery'].create_module(template='optic_fibre')

## Adding ephys

The second action is an extracellular electrophysiology recording

In [ ]:
project.actions['ecephys_1'] = {
    'type': 'Recording',
    'users': ['Sarah'],
    'tags': ['Open field']
}

In [ ]:
project.actions

In [ ]:
project.actions['ecephys_1'].attributes

In [ ]:
project.actions['ecephys_1'].tags.append('opto stim')

In [ ]:
project.actions

In [ ]:
ecephys_1 = project.actions['ecephys_1']

In [ ]:
ecephys_1.users, ecephys_1.type, ecephys_1.tags

## Adding a message to recording

In [ ]:
ecephys_1.create_message('Successful recording, albeit a bit noisy', user='Sarah')

In [ ]:
ecephys_1.messages

## Add probe location module

In [ ]:
ecephys_1.modules['probe_location'] = {
    'v1l': {
        'probe_0': {
            'value': 1000,
            'unit': 'um'
        }
    }
}

In [ ]:
ecephys_1.modules

## Adding data to action

In [ ]:
ecephys_1.data['main'] = 'ecephys_1.exdir'
#ecephys_1.data['main'] = 'ecephys_1.nwb'

In [ ]:
ecephys_1.data_path('main')

We use shutil to move the file, it's also possible to move it manually

In [ ]:
shutil.copytree('ecephys_1.exdir' , ecephys_1.data_path('main'))
#shutil.copyfile('ecephys_1.nwb' , ecephys_1.data_path('main'))

## Assignment - add a new action called ecphys_2

In [ ]:
project.actions['ecephys_2'] = {
    'type': 'Recording',
    'users': ['Mikkel'],
    'tags': ['Open field']
}

project.actions['ecephys_2'].data['main'] = 'ecephys_2.nwb'
project.actions['ecephys_2'].create_message('Fantastic recording!', user='Mikkel')
shutil.copyfile('ecephys_2.nwb' , project.actions['ecephys_2'].data_path('main'))

In [ ]:
project.actions

## We forgot to add entities in the recordings

In [ ]:
for action in project.actions.values():
    if action.type == 'Recording':
        action.entities.append('rat_180')

In [ ]:
project.actions

## Creating a template

In [ ]:
probe_location = {
    'identifier': 'probe_location',
    'description': 'v1l, v1r represents primary visual cortex left and right respectively.',
    'v1l': {
        'probe_0': ''
    },
    'v1r': {
        'probe_0': ''
    }
}
project.create_template('probe_location', contents=probe_location)

In [ ]:
probe_location_2 = project.actions['ecephys_2'].create_module(template='probe_location')

In [ ]:
probe_location_2['v1l']['probe_0'] = 1000 * pq.um

In [ ]:
probe_location_2['v1r']['probe_0'] = 1000 * pq.um

In [ ]:
project.actions['ecephys_2'].modules

## Using the browser

In [ ]:
expipe.Browser(project.path).display()

In [ ]:
df = pd.read_csv(project.actions['all'].data_path('sessions'))
df

# Do analysis on septum mec dataframe and save as action

# Interact with Github